# Medical Robotics - Week 1 Training

**Project**: Physics-Informed Foundation Models for Medical Robotics  
**Goal**: Train PPO, GNS, and PhysRobot on PushBox task  
**Target**: 12.5x sample efficiency, >95% OOD generalization  

**GitHub**: https://github.com/zhuangzard/medical-robotics-sim  
**Generated**: 2026-02-05 15:09:47  

---

## ⚠️ Setup Required

1. **Runtime**: Change to GPU (Runtime → Change runtime type)
2. **GPU Type**: Select V100 or A100 (Colab Pro)
3. **Run All**: Runtime → Run all

---

In [ ]:
# 🔍 GPU Detection
import subprocess
import torch

print('='*60)
print('🎮 GPU Configuration')
print('='*60)

# Check GPU
try:
    gpu_info = subprocess.check_output(
        ['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader']
    ).decode('utf-8').strip()
    print(f'GPU: {gpu_info}')
except:
    print('❌ No GPU detected! Please change runtime to GPU.')

# PyTorch check
print(f'PyTorch: {torch.__version__}')
print(f'CUDA Available: {torch.cuda.is_available()}')

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'GPU Memory: {gpu_mem:.1f} GB')
    
    # Auto-configure
    if 'A100' in gpu_name:
        batch_size, workers = 128, 8
        print('🚀 A100 detected: batch_size=128, workers=8')
    elif 'V100' in gpu_name:
        batch_size, workers = 64, 4
        print('🚀 V100 detected: batch_size=64, workers=4')
    else:
        batch_size, workers = 32, 2
        print('🚀 T4 detected: batch_size=32, workers=2')
else:
    batch_size, workers = 16, 2
    print('⚠️  CPU mode (slow!)')

print('='*60)

In [ ]:
# 📦 Install Dependencies
print('Installing dependencies...')

!pip install -q torch torchvision torchaudio
!pip install -q torch-geometric
!pip install -q gymnasium mujoco
!pip install -q stable-baselines3
!pip install -q matplotlib numpy scipy tqdm

print('✅ Dependencies installed!')

In [ ]:
# 📥 Clone Project Repository
import os
from pathlib import Path

REPO_URL = 'https://github.com/zhuangzard/medical-robotics-sim'
REPO_NAME = 'medical-robotics-sim'

if not Path(REPO_NAME).exists():
    print(f'Cloning {REPO_NAME}...')
    !git clone {REPO_URL}
    print('✅ Repository cloned')
else:
    print(f'{REPO_NAME} exists, pulling latest...')
    %cd {REPO_NAME}
    !git pull
    %cd ..

%cd {REPO_NAME}
print(f'\n📂 Working directory: {os.getcwd()}')
!ls -la

In [ ]:
# 📊 Setup Progress Tracking
from google.colab import drive
import json
from datetime import datetime
from pathlib import Path
import os

# Mount Drive (with error handling)
try:
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
        print('✅ Drive mounted')
    else:
        print('ℹ️  Drive already mounted')
except Exception as e:
    print(f'⚠️  Drive mount failed: {e}')
    print('Continuing without Drive sync...')
    # Create local fallback directory
    Path('/content/progress').mkdir(exist_ok=True)

# Create progress directory (Drive or local fallback)
if os.path.exists('/content/drive/MyDrive'):
    progress_dir = Path('/content/drive/MyDrive/medical-robotics-progress')
    progress_dir.mkdir(parents=True, exist_ok=True)
    print(f'📂 Progress dir: {progress_dir} (Drive)')
else:
    progress_dir = Path('/content/progress')
    progress_dir.mkdir(parents=True, exist_ok=True)
    print(f'📂 Progress dir: {progress_dir} (local fallback)')

progress_file = progress_dir / 'training_progress.json'

def update_progress(status, **kwargs):
    """Update progress file in Drive"""
    progress = {
        'timestamp': datetime.now().isoformat(),
        'status': status,
        'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU',
        **kwargs
    }
    with open(progress_file, 'w') as f:
        json.dump(progress, f, indent=2)
    print(f'📊 Progress updated: {status}')

update_progress('started', message='Training initialization complete')

print(f'✅ Progress tracking setup')
print(f'📁 Progress file: {progress_file}')

In [ ]:
# 🚀 Start Training
import time

print('='*60)
print(f'🏁 Training Started: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
print('='*60)

start_time = time.time()

try:
    # Update progress
    update_progress('training', message='Training in progress', eta_hours=8)
    
    # Run training script
    !bash experiments/week1_push_box/setup_and_run.sh
    
    # Training complete
    duration_sec = time.time() - start_time
    duration_hr = duration_sec / 3600
    
    update_progress('complete', 
                   message='Training complete', 
                   duration_hours=duration_hr)
    
    print('='*60)
    print('✅ Training Complete!')
    print(f'⏱️  Duration: {duration_hr:.1f} hours')
    print('='*60)
    
except Exception as e:
    update_progress('error', message=str(e))
    print(f'❌ Error: {e}')
    raise

In [ ]:
# 💾 Save Results to Drive
import shutil
import os

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Try Drive first, fallback to /tmp (避免递归)
if os.path.exists('/content/drive/MyDrive'):
    results_dir = Path(f'/content/drive/MyDrive/medical-robotics-results/{timestamp}')
    save_location = 'Google Drive'
else:
    results_dir = Path(f'/tmp/medical-robotics-results/{timestamp}')
    save_location = '/tmp (local)'

print(f'💾 Saving to: {save_location}')

results_dir.mkdir(parents=True, exist_ok=True)

print(f'📁 Saving results to: {results_dir}')

# Copy directories (exclude timestamp dirs to avoid recursion)
import re
dirs_to_save = ['results', 'models', 'data']

def ignore_results_dirs(dir_path, names):
    """Ignore timestamp-based result directories to prevent recursion"""
    # Ignore directories matching YYYYMMDD_HHMMSS pattern
    return [n for n in names if re.match(r'\d{8}_\d{6}', n)]

for dir_name in dirs_to_save:
    src = Path(dir_name)
    if src.exists() and src.is_dir():
        dst = results_dir / dir_name
        try:
            shutil.copytree(src, dst, 
                          ignore=ignore_results_dirs,
                          dirs_exist_ok=True)
            print(f'  ✅ {dir_name}/ ({sum(1 for _ in dst.rglob("*"))} files)')
        except Exception as e:
            print(f'  ⚠️  {dir_name}/ skipped: {e}')

# Copy key files
files_to_save = ['*.json', '*.md', '*.png', '*.pdf', '*.tex']
saved_count = 0
for pattern in files_to_save:
    for file in Path('.').glob(pattern):
        if file.is_file():
            shutil.copy2(file, results_dir / file.name)
            saved_count += 1

print(f'  ✅ {saved_count} files')

# Create summary
summary = {
    'timestamp': timestamp,
    'duration_hours': duration_hr,
    'gpu': torch.cuda.get_device_name(0),
    'status': 'complete',
    'drive_path': str(results_dir)
}

with open(results_dir / 'summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

update_progress('saved', message='Results saved to Drive', path=str(results_dir))

print(f'\n✅ All results saved to Drive!')
print(f'📂 {results_dir}')

In [ ]:
# 📊 Display Results
from IPython.display import Image, Markdown, display

print('='*60)
print('📊 Week 1 Training Results')
print('='*60)

# Display Table 1
table1_path = 'results/tables/sample_efficiency.md'
if Path(table1_path).exists():
    print('\n📋 Table 1: Sample Efficiency Comparison\n')
    with open(table1_path) as f:
        display(Markdown(f.read()))

# Display Figure 2
fig2_path = 'results/figures/ood_generalization.png'
if Path(fig2_path).exists():
    print('\n📈 Figure 2: OOD Generalization\n')
    display(Image(fig2_path))

# Display final report
report_path = 'results/WEEK1_FINAL_REPORT.md'
if Path(report_path).exists():
    print('\n📄 Final Report (excerpt):\n')
    with open(report_path) as f:
        lines = f.readlines()[:50]  # First 50 lines
        display(Markdown(''.join(lines)))

print('\n✅ Training Complete! All results saved to Drive.')